<a href="https://colab.research.google.com/github/FadyRafat0/Car-Price-Prediction/blob/main/%D9%90AI_PROJECT_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Car Price Prediction**

# Column description


**Car Id** : represent the id for each car where each car has unique id

**Levy**: A levy is a legal seizure of your property to satisfy a tax debt

**Manufacturer**: The brand that manfactured the car

**Model**: The model of the car

**Prod_year** : The year the car producted in

**Category**: Category of the car As ( jeep , car , hatchback , etc..)

**Leather interior** : is the car interior as the chairs made of leather or not

**Feul type** : The type of the feul that the car takes

**Engine Volume** : The volume the car takes of feul

**Mileage** : The moved Mileage by the car

**Cylinders** : how many cylinders the car contains where by increasng the car becomes much better

**Gear box type** : Descripe whether the car is automatic or manual

**Drive wheels** : Determine whether the car moves by the front tires or the backward tires or by both as jeep cars

**Doors** : Determine the number of the doors in the car

**Color** : Determine the color of the car

**Airbags** : The number of Air bags the car contains

# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeRegressor

from xgboost import XGBRegressor

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset,Dataset

# feature selection
from sklearn.feature_selection import RFE, RFECV
import pickle
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestRegressor


# Load Dataset

In [ ]:
train_df = pd.read_csv('train_data.csv')
test_df = pd.read_csv('test_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train_data.csv'

#EDA and Data Visualization

## First Look

In [ ]:
test_df.head()

In [ ]:
test_df.tail()

## Understanding the Data

In [ ]:
train_df.describe()

In [ ]:
test_df.describe()

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
train_df.shape

In [ ]:
test_df.shape

###Visualize Categorical Data

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(x='Manufacturer', data=train_df)
plt.xticks(rotation=90)
plt.title('Distribution of Car Manufacturers')
plt.xlabel('Manufacturer')
plt.ylabel('Count')
plt.show()


#### From this Graph We conclude that Toyota Cars Has the Most Manfuctured Cars in our dataset

In [ ]:
plt.figure(figsize=(30, 30))
train_df['Category'].value_counts().plot.pie(autopct='%1.1f%%')
plt.title('Distribution of Car Categories')
plt.ylabel('')
plt.show()


#### From this pie chart we conclude that Sedan is the most Percentage category in our data followed by Jeep

In [ ]:
leather_counts = train_df['Leather interior'].value_counts()

plt.figure(figsize=(8, 8))
plt.pie(leather_counts, labels=leather_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Distribution of Leather Interior')
plt.axis('equal')
plt.show()


#### from the pie chart we conclude that 72% of the cars have Leather interior and this to high compared to the cars that dont have

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Fuel type', data=train_df)
plt.title('Distribution of Fuel Types')
plt.xlabel('Fuel Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


#### From this barchart we conclude that Petrol is the most used Feul type in our data

In [ ]:
gearbox_counts = train_df['Gear box type'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(gearbox_counts, labels=gearbox_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Distribution of Gear Box Types')
plt.axis('equal')


#### From this pie chart we conclude that 70% of the cars in our data is Automatic cars followed by tiptroinic cars

In [ ]:
drive_wheels_counts = train_df['Drive wheels'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(drive_wheels_counts, labels=drive_wheels_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Distribution of Drive Wheels')
plt.axis('equal')
plt.show()


###Visualize Numerical data

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(train_df['Levy'], kde=True)
plt.title('Distribution of Levy')
plt.xlabel('Levy')
plt.ylabel('Frequency')
plt.show()


#### The distribution of the Levy values is heavily skewed to the right, with a significant number of vehicles having either very low or zero levy. This suggests that most cars in the dataset come with minimal or no extra cost/tax. However, there are a few cars with very high levy values, which appear to be outliers. These outliers might represent premium or luxury vehicles, or they could indicate data entry errors that warrant further inspection.





In [ ]:
plt.figure(figsize=(30, 10))
sns.histplot(train_df['Prod. year'], kde=True)
plt.title('Distribution of Production Year')
plt.xlabel('Production Year')
plt.ylabel('Frequency')
plt.show()

#### From This BarGraph We conclude That The most Year With Cars Manfuctrued is 2022

In [ ]:
plt.figure(figsize=(40, 30))
sns.histplot(train_df['Engine volume'], kde=True)
plt.title('Distribution of Engine Volume')
plt.xlabel('Engine Volume')
plt.ylabel('Frequency')
plt.show()


#### The distribution of engine volume is right-skewed, with the majority of vehicles having an engine volume between 1.5L and 3.0L, which is typical for standard consumer cars. The peak appears around 2.0L, indicating that it's the most common engine size.

# PreProcessing


## Function that combine Preprocessing



In [ ]:
def preprocess_data(df):
    """
    Preprocesses the input DataFrame by cleaning and validating each column.
    Prints unique values, invalid entries, and counts for each column.
    Returns the cleaned DataFrame.
    """
    print("\n" + "="*50 + "\n")

    # === 'ID' ===
    print("Unique values for 'ID':")
    print(df['ID'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Price' ===
    print("Unique values for 'Price':")
    print(df['Price'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Levy' ===
    print("Unique values for 'Levy':")
    print(df['Levy'].unique())
    Nonvalue_levy = df[df['Levy'] == '-']
    print("'Levy' cells that have no value:", Nonvalue_levy['Levy'].unique())
    print(f"Number of non-value cells in 'Levy': {Nonvalue_levy.shape[0]}")

    # Replace cells that have '-' with Nulls
    df['Levy'] = df['Levy'].replace('-', np.nan)
    df['Levy'] = df['Levy'].astype(float)

    print("Unique values for 'Levy' after cleaning:")
    print(df['Levy'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Manufacturer' ===
    print("Unique values for 'Manufacturer':")
    print(df['Manufacturer'].unique())
    invalid_manufacturers = df[~df['Manufacturer'].astype(str).apply(lambda x: x.isascii())]
    print("Invalid 'Manufacturer' values:", invalid_manufacturers['Manufacturer'].unique())
    print(f"Number of invalid 'Manufacturer' values: {invalid_manufacturers.shape[0]}")

    # Deleting rows with invalid manufacturers
    df = df[df['Manufacturer'].apply(lambda x: x.isascii() if pd.notnull(x) else True)]

    print("Unique values for 'Manufacturer' after cleaning:")
    print(df['Manufacturer'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Model' ===
    print("Unique values for 'Model':")
    print(df['Model'].unique())
    invalid_models = df[~df['Model'].astype(str).apply(lambda x: x.isascii()) | df['Model'].astype(str).str.isdigit()]
    print("Invalid 'Model' values:", invalid_models['Model'].unique())
    print(f"Number of invalid 'Model' values: {invalid_models.shape[0]}")

    # Remove rows with invalid models
    df = df[df['Model'].apply(lambda x: x.isascii() and not str(x).isdigit() if pd.notnull(x) else True)]

    print("Unique values for 'Model' after cleaning:")
    print(df['Model'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Production Year' ===
    print("Unique values for 'Prod. year':")
    print(df['Prod. year'].unique())
    invalid_prod_year = df['Prod. year'].apply(lambda x: pd.notnull(x) and not str(x).isdigit())
    print("Non-numeric 'Prod. year' entries:")
    print(df.loc[invalid_prod_year, 'Prod. year'].unique())
    print("Count of non-numeric 'Prod. year' entries:", invalid_prod_year.sum())

    # Removing rows with invalid production year
    df = df[~invalid_prod_year]
    df['Prod. year'] = df['Prod. year'].astype(float)

    print("Unique values for 'Prod. year' after cleaning:")
    print(df['Prod. year'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Category' ===
    print("Unique values for 'Category':")
    print(df['Category'].unique())
    count = df['Category'].isin(['Yes', 'No']).sum()
    print("Number of rows with 'Yes' or 'No' in 'Category':", count)

    # Remove rows with invalid category entries
    df = df[~df['Category'].isin(['Yes', 'No'])]

    print("Unique values for 'Category' after cleaning:")
    print(df['Category'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Leather Interior' ===
    print("Unique values for 'Leather interior':")
    print(df['Leather interior'].unique())
    count = df['Leather interior'].isin(['Yes', 'No']).sum()
    print("Number of rows with 'Yes' or 'No' in 'Leather interior':", count)

    # Remove rows with entries other than 'Yes' or 'No'
    df = df[df['Leather interior'].isin(['Yes', 'No']) | df['Leather interior'].isna()]

    print("Unique values for 'Leather interior' after cleaning:")
    print(df['Leather interior'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Fuel Type' ===
    print("Unique values for 'Fuel type':")
    print(df['Fuel type'].unique())
    invalid_fuel_types = df[~df['Fuel type'].astype(str).apply(lambda x: x.isalpha() or x == 'Plug-in Hybrid')]
    print("Invalid 'Fuel type' values:", invalid_fuel_types['Fuel type'].unique())
    print(f"Number of invalid 'Fuel type' values: {invalid_fuel_types.shape[0]}")

    # Remove invalid fuel type entries
    df = df[df['Fuel type'].apply(lambda x: x.isalpha() or x == 'Plug-in Hybrid' if pd.notnull(x) else True)]

    print("Unique values for 'Fuel type' after cleaning:")
    print(df['Fuel type'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Engine Volume' ===
    print("Unique values for 'Engine volume':")
    print(df['Engine volume'].unique())
    km_mask = df['Engine volume'].apply(lambda x: isinstance(x, str) and 'km' in x.lower())
    print("Values with 'km' in 'Engine volume':", df.loc[km_mask, 'Engine volume'].unique())
    print("Count of values with 'km' in 'Engine volume':", km_mask.sum())

    # Clean 'Turbo' values and remove 'km' entries
    df = df[~km_mask | df['Engine volume'].isna()]
    df['Engine volume'] = df['Engine volume'].apply(lambda x: x.split()[0] if isinstance(x, str) and 'Turbo' in x else x)
    df['Engine volume'] = df['Engine volume'].astype(float)

    print("Unique values for 'Engine volume' after cleaning:")
    print(df['Engine volume'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Mileage' ===
    print("Unique values for 'Mileage':")
    print(df['Mileage'].unique())
    invalid_mileage = df[~df['Mileage'].astype(str).str.strip().str.endswith('km') & df['Mileage'].notna()]
    print("Invalid 'Mileage' values:", invalid_mileage['Mileage'].unique())
    print("Number of invalid 'Mileage' values:", invalid_mileage.shape[0])

    # Keep only valid 'km' entries
    valid_km_mask = df['Mileage'].astype(str).str.strip().str.endswith('km')
    df = df[valid_km_mask | df['Mileage'].isna()]
    df['Mileage'] = df['Mileage'].astype(str)
    df['Mileage'] = df['Mileage'].str.extract(r'(\d+)')
    df['Mileage'] = df['Mileage'].astype(float)

    print("Unique values for 'Mileage' after cleaning:")
    print(df['Mileage'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Cylinders' ===
    print("Unique values for 'Cylinders':")
    print(df['Cylinders'].unique())
    strange_cylinders = df[~df['Cylinders'].astype(str).str.isdigit() & df['Cylinders'].notna()]
    print("Invalid 'Cylinders' values:", strange_cylinders['Cylinders'].unique())
    print(f"Number of invalid 'Cylinders' values: {strange_cylinders.shape[0]}")

    # Remove rows with invalid cylinder entries
    df = df[df['Cylinders'].astype(str).str.isdigit() | df['Cylinders'].isna()]
    df['Cylinders'] = df['Cylinders'].astype(float)

    print("Unique values for 'Cylinders' after cleaning:")
    print(df['Cylinders'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Gear Box Type' ===
    print("Unique values for 'Gear box type':")
    print(df['Gear box type'].unique())
    valid_gearbox_types = ['Automatic', 'Manual', 'Tiptronic', 'Variator']
    invalid_gearbox = df[~df['Gear box type'].isin(valid_gearbox_types) & df['Gear box type'].notnull()]
    print("Invalid 'Gear box type' values:", invalid_gearbox['Gear box type'].unique())
    print(f"Number of invalid 'Gear box type' values: {invalid_gearbox.shape[0]}")

    # Remove rows with invalid gearbox entries
    df = df[df['Gear box type'].isin(valid_gearbox_types) | df['Gear box type'].isna()]

    print("Unique values for 'Gear box type' after cleaning:")
    print(df['Gear box type'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Drive Wheels' ===
    print("Unique values for 'Drive wheels':")
    print(df['Drive wheels'].unique())
    valid_drive_wheels = ['Front', 'Rear', '4x4']
    invalid_drive_wheels = df[~df['Drive wheels'].isin(valid_drive_wheels) & df['Drive wheels'].notnull()]
    print("Invalid 'Drive wheels' values:", invalid_drive_wheels['Drive wheels'].unique())
    print(f"Number of invalid 'Drive wheels' values: {invalid_drive_wheels.shape[0]}")

    # Remove rows with invalid drive wheel entries
    df = df[df['Drive wheels'].isin(valid_drive_wheels) | df['Drive wheels'].isna()]

    print("Unique values for 'Drive wheels' after cleaning:")
    print(df['Drive wheels'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Doors' ===
    print("Unique values for 'Doors':")
    print(df['Doors'].unique())
    # Drop the 'Doors' column since it doesn't have meaningful values
    df.drop('Doors', axis=1, inplace=True)
    print("'Doors' column dropped")
    print("\n" + "-"*50 + "\n")

    # === 'Wheel' ===
    print("Unique values for 'Wheel':")
    print(df['Wheel'].unique())
    valid_wheels = ['Left wheel', 'Right-hand drive']
    invalid_wheels = df[~df['Wheel'].isin(valid_wheels) & df['Wheel'].notnull()]
    print("Invalid 'Wheel' values:", invalid_wheels['Wheel'].unique())
    print(f"Number of invalid 'Wheel' values: {invalid_wheels.shape[0]}")

    # Remove rows with invalid wheel entries
    df = df[df['Wheel'].isin(valid_wheels) | df['Wheel'].isna()]

    print("Unique values for 'Wheel' after cleaning:")
    print(df['Wheel'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Color' ===
    print("Unique values for 'Color':")
    print(df['Color'].unique())
    invalid_colors = df[df['Color'].astype(str).str.isdigit()]
    print("Invalid 'Color' values:", invalid_colors['Color'].unique())
    print(f"Number of invalid 'Color' values: {invalid_colors.shape[0]}")

    # Remove rows with invalid color entries
    df = df[~df['Color'].astype(str).str.isdigit() | df['Color'].isna()]

    print("Unique values for 'Color' after cleaning:")
    print(df['Color'].unique())
    print("\n" + "-"*50 + "\n")

    # === 'Airbags' ===
    print("Unique values for 'Airbags':")
    print(df['Airbags'].unique())
    print("\n" + "="*50 + "\n")

    return df

In [ ]:
train_cleaned = preprocess_data(train_df)
test_cleaned = preprocess_data(test_df)


In [ ]:
train_cleaned.info()

In [ ]:
test_cleaned.info()

## Outliers handling


In [ ]:
def cap_outliers_iqr(dataframe, columns):
    for col in columns:
        if pd.api.types.is_numeric_dtype(dataframe[col]):
            Q1 = dataframe[col].quantile(0.25)
            Q3 = dataframe[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            dataframe[col] = dataframe[col].clip(lower=lower_bound, upper=upper_bound)
    return dataframe

train_cleaned = cap_outliers_iqr(train_cleaned, train_cleaned.columns)
test_cleaned = cap_outliers_iqr(test_cleaned, test_cleaned.columns)

## Nulls handling


In [ ]:
def fill_missing_values(df):
    for column in df.columns:
        if df[column].isnull().any():
            if df[column].dtype == 'object' or df[column].dtype.name == 'category':
                mode = df[column].mode()
                if not mode.empty:
                    df[column].fillna(mode[0], inplace=True)
            else:
                median = df[column].median()
                df[column].fillna(median, inplace=True)
    return df



In [ ]:
train_cleaned = fill_missing_values(train_cleaned)
test_cleaned = fill_missing_values(test_cleaned)

In [ ]:
train_cleaned.isna().sum()

In [ ]:
test_cleaned.isna().sum()

###Copy CSV File

In [ ]:
train_cleaned.to_csv('train_cleaned.csv', index=False)
test_cleaned.to_csv('test_cleaned.csv', index=False)

## Encoding

In [ ]:
Le = LabelEncoder()
for col in train_cleaned.columns:
    if train_cleaned[col].dtype == 'object':
        train_cleaned[col] = Le.fit_transform(train_cleaned[col])

Le = LabelEncoder()
for col in test_cleaned.columns:
    if test_cleaned[col].dtype == 'object':
        test_cleaned[col] = Le.fit_transform(test_cleaned[col])


###Basic info

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

print("\n" + "="*80)
print("CLEANED DATA EXPLORATORY ANALYSIS")
print("="*80 + "\n")


print("Cleaned Train Dataset Shape:", train_cleaned.shape)
print("Cleaned Test Dataset Shape:", test_cleaned.shape)


print(train_cleaned.head())
print(test_cleaned.head())


print(train_cleaned.dtypes)
print(test_cleaned.dtypes)


print(train_cleaned.describe().transpose())
print(test_cleaned.describe().transpose())


print(train_cleaned.isna().sum())
print(test_cleaned.isna().sum())

###Data Distribution

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(train_cleaned['Price'], kde=True, bins=50, color='darkblue')
plt.title('Distribution of Car Prices (Cleaned Data)', fontsize=15)
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


# Correlation matrix of cleaned data
plt.figure(figsize=(14, 12))
correlation_matrix = train_cleaned.corr()
mask = np.triu(correlation_matrix)
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            mask=mask, linewidths=0.5)
plt.title('Correlation Matrix (Cleaned Data)', fontsize=16)
plt.tight_layout()
plt.show()

# Feature correlations with Price
price_corr = correlation_matrix['Price'].sort_values(ascending=False)
plt.figure(figsize=(12, 8))
sns.barplot(x=price_corr.values, y=price_corr.index, palette='viridis')
plt.title('Feature Correlations with Price (Cleaned Data)', fontsize=15)
plt.xlabel('Correlation Coefficient')
plt.tight_layout()
plt.show()

In [ ]:
# Production Year Analysis
plt.figure(figsize=(12, 6))
sns.scatterplot(x='Prod. year', y='Price', data=train_cleaned, alpha=0.6, color='darkblue')
plt.title('Price vs Production Year (Cleaned Data)', fontsize=15)
plt.xlabel('Production Year')
plt.ylabel('Price')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Engine volume analysis
plt.figure(figsize=(12, 6))
sns.scatterplot(x='Engine volume', y='Price', data=train_cleaned, alpha=0.6, color='darkgreen')
plt.title('Price vs Engine Volume (Cleaned Data)', fontsize=15)
plt.xlabel('Engine Volume')
plt.ylabel('Price')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Mileage analysis
plt.figure(figsize=(12, 6))
sns.scatterplot(x='Mileage', y='Price', data=train_cleaned, alpha=0.6, color='indianred')
plt.title('Price vs Mileage (Cleaned Data)', fontsize=15)
plt.xlabel('Mileage')
plt.ylabel('Price')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Analyze top manufacturers by price
plt.figure(figsize=(14, 8))
top_manufacturers = train_cleaned.groupby('Manufacturer')['Price'].mean().sort_values(ascending=False).head(10)
sns.barplot(x=top_manufacturers.index, y=top_manufacturers.values, palette='Set3')
plt.title('Top 10 Manufacturers by Average Price (Cleaned Data)', fontsize=15)
plt.xticks(rotation=45, ha='right')
plt.ylabel('Average Price')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Fuel type analysis
plt.figure(figsize=(12, 8))
sns.boxplot(x='Fuel type', y='Price', data=train_cleaned, palette='Set2')
plt.title('Price Distribution by Fuel Type (Cleaned Data)', fontsize=15)
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Gear box type analysis
plt.figure(figsize=(12, 8))
sns.boxplot(x='Gear box type', y='Price', data=train_cleaned, palette='Set2')
plt.title('Price Distribution by Gear Box Type (Cleaned Data)', fontsize=15)
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Leather interior analysis
plt.figure(figsize=(10, 6))
sns.boxplot(x='Leather interior', y='Price', data=train_cleaned, palette='Set2')
plt.title('Price Distribution by Leather Interior (Cleaned Data)', fontsize=15)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Drive wheels analysis
plt.figure(figsize=(12, 6))
sns.boxplot(x='Drive wheels', y='Price', data=train_cleaned, palette='Set2')
plt.title('Price Distribution by Drive Wheels (Cleaned Data)', fontsize=15)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Cylinders analysis
plt.figure(figsize=(12, 6))
sns.boxplot(x='Cylinders', y='Price', data=train_cleaned, palette='Set2')
plt.title('Price Distribution by Cylinders (Cleaned Data)', fontsize=15)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Create pairplot for main numerical features
numerical_cols = ['Price', 'Prod. year', 'Engine volume', 'Mileage', 'Cylinders']
plt.figure(figsize=(15, 15))
sample_df = train_cleaned[numerical_cols].sample(min(1000, len(train_cleaned)))
sns.pairplot(sample_df, diag_kind='kde', plot_kws={'alpha': 0.6})
plt.suptitle('Pairplot of Main Numerical Features (Cleaned Data)', y=1.02, fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Distribution of top categorical features after encoding
categorical_features = ['Manufacturer', 'Model', 'Category', 'Fuel type', 'Gear box type', 'Drive wheels']
plt.figure(figsize=(16, 12))
for i, feature in enumerate(categorical_features):
    plt.subplot(3, 2, i+1)
    sns.histplot(train_cleaned[feature], kde=True, color='darkblue')
    plt.title(f'Distribution of {feature} After Encoding', fontsize=12)
    plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()




## Assign features and target variable to be used in Modeling





In [ ]:
x_train = train_cleaned.drop('Price', axis=1)
y_train = train_cleaned['Price']

x_test = test_cleaned.drop('Price', axis=1)
y_test = test_cleaned['Price']


# Modeling


## **Linear Regression**



### Take instances of the data

In [ ]:
X_train_LR = x_train
y_train_LR = y_train

X_test_LR = x_test
y_test_LR = y_test

### Scaling


In [ ]:
scaler = StandardScaler()
X_train_LR_scaled = scaler.fit_transform(X_train_LR)
X_test_LR_scaled = scaler.transform(X_test_LR)

### Feature Selection

In [ ]:
estimator = LinearRegression()
cv = KFold(n_splits=5)
selector = RFECV(estimator, step=1, cv=cv, scoring='neg_mean_squared_error')   # Recursive Feature Elimination with Cross-Validation
X_train_selected = selector.fit_transform(X_train_LR_scaled, y_train_LR)
X_test_selected = selector.transform(X_test_LR_scaled)

selected_features = X_train_LR.columns[selector.get_support()]
print("Selected Features:", selected_features)

### Create model


In [ ]:
model_LR_scaled = LinearRegression()

### Train model

In [ ]:
model_LR_scaled.fit(X_train_selected, y_train_LR)             # minimizing the cost function

### Predict


In [ ]:
y_pred_LR = model_LR_scaled.predict(X_test_selected)

### Evaluate

In [ ]:
mse_LR_scaled = mean_squared_error(y_test_LR, y_pred_LR)
mae_LR_scaled = mean_absolute_error(y_test_LR, y_pred_LR)
r2_LR_scaled = r2_score(y_test_LR, y_pred_LR)

print(f"Mean Square Error of Linear Regression: {mse_LR_scaled:.2f}")
print(f"Mean Absolute Error of Linear Regression: {mae_LR_scaled:.2f}")
print(f"R² Score of Linear Regression: {r2_LR_scaled:.2f}")


### Plot Actual vs Predicted


In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_test_LR, y_pred_LR, color='blue', alpha=0.6)
plt.plot([y_test_LR.min(), y_test_LR.max()], [y_test_LR.min(), y_test_LR.max()], 'r--', lw=2)
plt.xlabel('Actual Prices using Linear Regression')
plt.ylabel('Predicted Prices using Linear Regression')
plt.title('Actual vs Predicted Car Prices')
plt.show()

##**Polynomial** **Regression**

### Feature Selection

In [ ]:
plt.figure(figsize=(10, 8))
corr_matrix = train_cleaned.corr(numeric_only=True)
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title("Feature Correlation Heatmap")
plt.tight_layout()
plt.show()

In [ ]:
price_correlations = corr_matrix['Price'].drop('Price').abs().sort_values(ascending=False)
print("Features most correlated with Price:")
print(price_correlations)


#select top features
top_features = price_correlations.head(10).index.tolist()
print(f"Selected top features: {top_features}")

In [ ]:
X_train_P = x_train[top_features]  # Use selected features
y_train_P = y_train
X_test_P = x_test[top_features]  # Use selected features
y_test_P = y_test



scaler_P = StandardScaler()
X_train_P_scaled = scaler_P.fit_transform(X_train_P)
X_test_P_scaled = scaler_P.transform(X_test_P)


#Apply polynomial transformation (after scaling)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_P_scaled)
X_test_poly = poly.transform(X_test_P_scaled)


print(f"Original features count: {X_train_P.shape[1]}")
print(f"Polynomial features count: {X_train_poly.shape[1]}")


### Create and fit the Linear Regression model


In [ ]:
model_poly = LinearRegression()
model_poly.fit(X_train_poly, y_train_P)


###Cross Validation to check overfitting

In [ ]:
cv_scores = cross_val_score(model_poly, X_train_poly, y_train_P,
                           cv=5, scoring='neg_mean_squared_error')
print(f"Cross-validation MSE scores: {-cv_scores}")
print(f"Average CV MSE: {-cv_scores.mean()}")

###**Predict**

In [ ]:
y_pred_poly = model_poly.predict(X_test_poly)


### Evaluate the model

In [ ]:
mse_poly = mean_squared_error(y_test_P, y_pred_poly)
r2_poly = r2_score(y_test_P, y_pred_poly)
print(f"Mean Squared Error: {mse_poly}")
print(f"R² Score: {r2_poly}")

###Analyze residuals

In [ ]:
residuals = y_test_P - y_pred_poly
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.scatter(y_pred_poly, residuals)
plt.axhline(y=0, color='r', linestyle='-')
plt.title('Residuals vs Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')

plt.subplot(1, 2, 2)
sns.histplot(residuals, kde=True)
plt.title('Distribution of Residuals')
plt.xlabel('Residual Value')

plt.tight_layout()
plt.show()

### Mileage Attribute

In [ ]:
if 'Mileage' in top_features:
    mileage_idx = top_features.index('Mileage')

    X_mean = X_train_P.mean().values.reshape(1, -1)
    X_mean_array = np.repeat(X_mean, 100, axis=0)

    mileage_range = np.linspace(train_cleaned['Mileage'].min(), train_cleaned['Mileage'].max(), 100)
    X_mean_array[:, mileage_idx] = mileage_range

    # Scale and transform
    X_mean_scaled = scaler_P.transform(X_mean_array)
    X_mean_poly = poly.transform(X_mean_scaled)


    y_mean_pred = model_poly.predict(X_mean_poly)

    # Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(train_cleaned['Mileage'], train_cleaned['Price'], color='blue', alpha=0.3, label='Actual data')
    plt.plot(mileage_range, y_mean_pred, color='red', linewidth=2, label='Polynomial fit (degree 2)')
    plt.title('Polynomial Regression: Mileage vs Price')
    plt.xlabel('Mileage')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()



## **SVR**

### Split into train/test sets


In [ ]:
x_train_SVR = x_train
y_train_SVR = y_train

x_test_SVR = x_test
y_test_SVR = y_test

### Scaling Data

In [ ]:
# Scale the features
scaler_X = StandardScaler()
X_train_SVR_scaled = scaler_X.fit_transform(x_train_SVR)
X_test_SVR_scaled = scaler_X.transform(x_test_SVR)

# Scale the target variable (y)
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train_SVR.values.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test_SVR.values.reshape(-1, 1)).ravel()



X_train_SVR_selected = X_train_SVR_scaled
X_test_SVR_selected = X_test_SVR_scaled
print(f"Using all {X_train_SVR_selected.shape[1]} features")


### Create the Support Vector Regression model


In [ ]:
svr_model = SVR()

### Tune SVR Hyperparameters


In [ ]:
# Define the parameter distribution for RandomizedSearchCV
param_dist_SVR = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': np.logspace(-3, 3, 10),
    'epsilon': np.logspace(-3, 0, 10),
    'gamma': ['scale', 'auto'] + list(np.logspace(-3, 0, 5)),
    'degree': [2, 3, 4]  # only used when kernel='poly'
}

#take sample
SAMPLE_SIZE = min(1000, len(X_train_SVR_scaled))
indices = np.random.choice(len(X_train_SVR_scaled), SAMPLE_SIZE, replace=False)
X_sample = X_train_SVR_scaled[indices]
y_sample = y_train_scaled[indices]
print(f"Using {SAMPLE_SIZE} samples for parameter search.")


random_search_SVR = RandomizedSearchCV(
    estimator= SVR(),
    param_distributions=param_dist_SVR,
    scoring='neg_mean_squared_error',
    n_iter=30,
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=2
)

print("Starting parameter search...")
random_search_SVR.fit(X_sample, y_sample)

# Get best parameters from RandomizedSearchCV
best_svr_model = random_search_SVR.best_estimator_
print(f"Best Parameters: {random_search_SVR.best_params_}")

### Train the best model using the best parameters found


In [ ]:
best_svr_model.fit(X_train_SVR_scaled, y_train_scaled)

###Cross_validation evaluation

In [ ]:
print("Performing cross-validation...")
cv_scores = cross_val_score(
    best_svr_model,
    X_train_SVR_scaled,
    y_train_scaled,
    cv=5,
    scoring='neg_mean_squared_error'
)

print(f"Cross-validation MSE scores: {-cv_scores}")
print(f"Average CV MSE: {-cv_scores.mean():.4f}, STD: {cv_scores.std():.4f}")

### Make predictions


In [ ]:
y_pred_scaled = best_svr_model.predict(X_test_SVR_scaled)
y_pred_SVR = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

### Evaluate the model


In [ ]:

mse_SVR = mean_squared_error(y_test_SVR, y_pred_SVR)
mae_SVR = mean_absolute_error(y_test_SVR, y_pred_SVR)
r2_SVR = r2_score(y_test_SVR, y_pred_SVR)

print(f"Mean Squared Error of SVR: {mse_SVR:.2f}")
print(f"Mean Absolute Error of SVR: {mae_SVR:.2f}")
print(f"R² Score: {r2_SVR:.2f}")

### Plot Actual vs Predicted


In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test_SVR, y_pred_SVR, color='blue', alpha=0.6)
plt.plot([y_test_SVR.min(), y_test_SVR.max()], [y_test_SVR.min(), y_test_SVR.max()], 'r--', lw=2)
plt.xlabel('Actual Prices using SVR')
plt.ylabel('Predicted Prices using SVR')
plt.title(f'Actual vs Predicted Car Prices (SVR)\nBest Params: {random_search_SVR.best_params_}')
plt.show()

## **XGBoost Algo**

In [ ]:
xgb = XGBRegressor()

### Tune the XGB Boost Hyperparameters

In [ ]:
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5, 6, 7, 8],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

rand_xgb = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid_xgb,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=3,
    return_train_score=True,
    n_iter=50,
    random_state=42
)

### Dividing data into train and test

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
x_train_Xgb_poly = poly.fit_transform(x_train)
x_test_Xgb_poly = poly.transform(x_test)

### Scale the Data

In [ ]:
scaler_X = StandardScaler()
X_train_Xgb_scaled = scaler_X.fit_transform(x_train_Xgb_poly)
X_test_Xgb_scaled = scaler_X.transform(x_test_Xgb_poly)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).ravel()

### Fit the xgb model on the data

In [ ]:
rand_xgb.fit(X_train_Xgb_scaled, y_train_scaled)

### Visualize the best Params

In [ ]:
print("Best Parameters:", rand_xgb.best_params_)
print("Best Model:", rand_xgb.best_estimator_)
print("Best Score (Neg MSE):", rand_xgb.best_score_)

### Use the best parameters

In [ ]:
rand_xgb.best_estimator_.fit(X_train_Xgb_scaled, y_train_scaled)

### Use best estmaitor to predict our data

In [ ]:
y_pred = rand_xgb.best_estimator_.predict(X_test_Xgb_scaled)

### Evalute the model

In [ ]:
mse = mean_squared_error(y_test_scaled, y_pred)
mae = mean_absolute_error(y_test_scaled, y_pred)
r2 = r2_score(y_test_scaled, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R² Score: {r2}")

## **Neural Networks using Pytorch**

### Create the custom dataset

In [ ]:
class RegressionDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x = torch.tensor(x_data, dtype=torch.float32)
        self.y = torch.tensor(y_data, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

### Scale the Poly data

In [ ]:
scaler = StandardScaler()
X_train_nn_scaled = scaler.fit_transform(x_train)
X_test_nn_scaled = scaler.transform(x_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).ravel()

### Constract the Dataset and the Dataloader

In [ ]:
train_dataset = RegressionDataset(X_train_nn_scaled ,y_train_scaled)
test_dataset = RegressionDataset(X_test_nn_scaled, y_test_scaled)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

### Form the Linear Regression class for our fully connected nn

In [ ]:
class LinearRegression(nn.Module):
    def __init__(self, input_dim):
        super(LinearRegression, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.net(x)


### call the model and Choose the Criterion and optimizer

In [ ]:
model = LinearRegression(X_train_nn_scaled.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Run the Train loop

In [ ]:
epochs = 100
for epoch in range(epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X.float())
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

### Eval the model

In [ ]:
model.eval()
with torch.no_grad():
    test_loss = 0
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X.float())
        test_loss += criterion(outputs, batch_y).item()
    print(f'Test Loss: {test_loss / len(test_loader):.4f}')

### Calc The MSE And the MAE and The R^2

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        all_preds.extend(outputs.squeeze().tolist())
        all_targets.extend(targets.squeeze().tolist())

all_preds = np.array(all_preds)
all_targets = np.array(all_targets)

mse = np.mean((all_preds - all_targets) ** 2)
mae = mean_absolute_error(all_targets, all_preds)
r2 = r2_score(all_targets, all_preds)

print(f"\n Evaluation Results:")
print(f" MSE:  {mse:.4f}")
print(f" MAE:  {mae:.4f}")
print(f" R² Score: {r2:.4f}")


## **Decision** **Tree**

### Random Search with Decision Tree

In [ ]:
model = DecisionTreeRegressor(random_state=42)

param_dist = {
    'max_depth': [3, 5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2', None]
}

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

random_search.fit(x_train, y_train)

### we will visualize the best parameter

In [ ]:
random_search.best_params_

In [ ]:
best_tree = random_search.best_estimator_

In [ ]:
poly = PolynomialFeatures(degree=2)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.transform(x_test)
y_train_poly = poly.fit_transform(y_train.values.reshape(-1, 1))
y_test_poly = poly.transform(y_test.values.reshape(-1, 1))

In [ ]:
best_tree.fit(x_train_poly, y_train_poly)

### we will evaluate the accuracy

In [ ]:
train_pred = best_tree.predict(x_train_poly)
test_pred = best_tree.predict(x_test_poly)

train_mse = mean_squared_error(y_train_poly, train_pred)
test_mse = mean_squared_error(y_test_poly, test_pred)

train_r2 = r2_score(y_train_poly, train_pred)
test_r2 = r2_score(y_test_poly, test_pred)

print("Train MSE:", train_mse)
print("Test MSE:", test_mse)

print("Train R^2:", train_r2)
print("Test R^2:", test_r2)

## **Random** **Forest**

In [ ]:
rf = RandomForestRegressor(random_state=42)

### define hyperparameters

In [ ]:
param_dist_rf = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

In [ ]:
random_search_rf = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist_rf,
    n_iter=20,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

### we are going to run our grid search

In [ ]:
random_search_rf.fit(x_train, y_train)

In [ ]:
random_search_rf.best_estimator_

In [ ]:
best_rf_model=random_search_rf.best_estimator_.fit(x_train_poly, y_train_poly)

In [ ]:
best_rf_model.predict(x_test_poly)

In [ ]:
train_predictions = best_rf_model.predict(x_train_poly)
test_predictions = best_rf_model.predict(x_test_poly)

train_mse = mean_squared_error(y_train_poly, train_predictions)
train_mae = mean_absolute_error(y_train_poly, train_predictions)
train_r2 = r2_score(y_train_poly, train_predictions)

test_mse = mean_squared_error(y_test_poly, test_predictions)
test_mae = mean_absolute_error(y_test_poly, test_predictions)
test_r2 = r2_score(y_test_poly, test_predictions)

print("Training Set:")
print(f"  MSE: {train_mse:.4f}")
print(f"  MAE: {train_mae:.4f}")
print(f"  R²: {train_r2:.4f}")

print("\nTest Set:")
print(f"  MSE: {test_mse:.4f}")
print(f"  MAE: {test_mae:.4f}")
print(f"  R²: {test_r2:.4f}")

## **Stacking** **using** **XGBoost**

###  Prepare Meta-Features and Targets

In [ ]:
X_meta = np.column_stack([
    y_pred_LR,
    y_pred_poly,
    y_pred_SVR,
    y_pred,
    all_preds,
    test_pred,
    test_predictions
])

y_meta = y_test

### Split meta data

In [ ]:
X_train_meta, X_test_meta, y_train_meta, y_test_meta = train_test_split(X_meta, y_meta, test_size=0.2, random_state=42)

 ### Define and tune meta model

In [ ]:
xgb_meta = XGBRegressor()

param_grid_meta = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

rand_meta_xgb = RandomizedSearchCV(
    estimator=xgb_meta,
    param_distributions=param_grid_meta,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=3,
    return_train_score=True,
    n_iter=50,
    random_state=42
)

rand_meta_xgb.fit(X_train_meta, y_train_meta)

### Evaluate Meta Model

In [ ]:
y_pred_meta = rand_meta_xgb.best_estimator_.predict(X_test_meta)

mse = mean_squared_error(y_test_meta, y_pred_meta)
mae = mean_absolute_error(y_test_meta, y_pred_meta)
r2 = r2_score(y_test_meta, y_pred_meta)

print("Best Parameters:", rand_meta_xgb.best_params_)
print("Best Score (Neg MSE):", rand_meta_xgb.best_score_)
print(f"Meta Model - MSE: {mse:.4f}")
print(f"Meta Model - MAE: {mae:.4f}")
print(f"Meta Model - R²: {r2:.4f}")

#GUI

In [ ]:
! pip install streamlit -q

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
! streamlit run app.py & npx localtunnel --port 8501
